In [1]:
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('~/Health/ozone_96_mort_est_total(1119).csv')

ozone = data[['loc','longitude','latitude',
              'arcgis_ID','location_id','cont_new_ID',
              'population_total_96','population_urban_96','population_rural_96',
              'MDA8h_urban_warm_96','MDA8h_rural_warm_96',
              'cvd_25_death_urban_est','cvd_25_death_urban_lb','cvd_25_death_urban_ub','cvd_25_death_rural_est','cvd_25_death_rural_lb','cvd_25_death_rural_ub']]
ozone = ozone.rename(columns={'cont_new_ID':'cont_ID'})

age_96 = pd.read_csv('~/ozone-mortality/age structure list/age_96_list.csv')
mort_96 = pd.read_csv('~/ozone-mortality/mortality list/mort_96_list.csv')
ozone_mort = ozone.merge(mort_96, how='left', left_on=['location_id'], right_on=['location_id'])
ozone_mort_age = ozone_mort.merge(age_96, how='left', left_on=['location_id','year','location_name'], right_on=['location_id','year','location_name'])

#### Here we prescribe all necessary parameters: multi-cause exposure-response RRs with 95% CIs, and the low-concentration cut-off (LCC, or threshold). The pooled RRs are estimated by meta-analysis (see a "tutorial" code file). The LCC value(s) are defined as the lowest 5 𝑡ℎ  percentile of the multi-data fused ambient ozone dataset (only considering the residential land areas) following Malley et al. 2017.

In [3]:
lnRR_cvd = np.log(1.017)
lnRR_sigma_cvd = (np.log(1.025) - np.log(1.017))/1.96 
LCC = 40

#### For the sake of quick computation (e.g. less than 10 minutes on JASMIN supercomputation system) as a demonstration, we have simplified some trivial aspects: (1) we cluster the age ≥25 population rather than 5-yr stratification. (2) we do not calculate the arithmetic mean and standard deviations, as we choose to report the median with 95% uncertainty intervals. 

In [4]:
data = ozone_mort_age
start = time.process_time()

for t in range(0,742299,1):
    
    ## Extract the age ≥25 risked proportion 
    pop_all_age       = data.iloc[t,58]
    pop_all_age_sigma = (data.iloc[t,58]-data.iloc[t,59])/1.96
    pop_25_age        = data.iloc[t,97]
    pop_25_age_sigma  = (data.iloc[t,97]-data.iloc[t,98])/1.96
    
    ## Extract the baseline mortality rate 
    cvd_25_mort            = data.iloc[t,19]
    cvd_25_mort_sigma      = (data.iloc[t,19]-data.iloc[t,20])/1.96
    
    ## Extract urban and rural population and MDA8h-ozone exposure 
    pop_urban_grid = data.iloc[t,7] 
    o3_mda8h_urban = data.iloc[t,9] 
    pop_rural_grid = data.iloc[t,8] 
    o3_mda8h_rural = data.iloc[t,10] 

    ## Convert into the hazadous dose from exposure concentration
    o3_mda8h_urban_dose = max(0,o3_mda8h_urban - LCC) 
    o3_mda8h_rural_dose = max(0,o3_mda8h_rural - LCC) 
    
    ## Generate Gaussian distributed samples 
    pop_all_age_dist      = np.random.normal(pop_all_age, pop_all_age_sigma, 1000) 
    pop_25_age_dist       = np.random.normal(pop_25_age,  pop_25_age_sigma,  1000)
    
    cvd_25_mort_dist      = np.random.normal(cvd_25_mort, cvd_25_mort_sigma, 1000)
    
    RR_dist_cvd           = np.random.normal(lnRR_cvd,    lnRR_sigma_cvd,    1000)
    
    ## Start the Monte Carlo bootstrap simulation realisations 
    delta_mort_cvd_urban  = cvd_25_mort_dist * (1 - np.exp(- RR_dist_cvd * o3_mda8h_urban_dose/10)) * pop_urban_grid * pop_25_age_dist / pop_all_age_dist / 100000 
    delta_mort_cvd_rural  = cvd_25_mort_dist * (1 - np.exp(- RR_dist_cvd * o3_mda8h_rural_dose/10)) * pop_rural_grid * pop_25_age_dist / pop_all_age_dist / 100000 
    
    ## Summarise the results    
    est_cvd_urban = np.nanpercentile(delta_mort_cvd_urban,50) 
    lb_cvd_urban  = np.nanpercentile(delta_mort_cvd_urban,2.5)
    ub_cvd_urban  = np.nanpercentile(delta_mort_cvd_urban,97.5)

    est_cvd_rural = np.nanpercentile(delta_mort_cvd_rural,50) 
    lb_cvd_rural  = np.nanpercentile(delta_mort_cvd_rural,2.5)
    ub_cvd_rural  = np.nanpercentile(delta_mort_cvd_rural,97.5)
    
    ## Store the distribution statistics into the DataFrame 
    data.iloc[t,11]  = est_cvd_urban
    data.iloc[t,12]  =  lb_cvd_urban
    data.iloc[t,13]  =  ub_cvd_urban

    data.iloc[t,14]  = est_cvd_rural
    data.iloc[t,15]  =  lb_cvd_rural
    data.iloc[t,16]  =  ub_cvd_rural
    
    if t%5000 == 0:
        print(t) ## Add a tracer to check the simulation progresses 
    
elapsed = (time.process_time() - start)
print('Time used:', elapsed, "s")    

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000
470000
475000
480000
485000
490000
495000
500000
505000
510000
515000
520000
525000
530000
535000
540000
545000
550000
555000
560000
565000
570000
575000
580000
585000
590000
595000
600000
605000
610000
615000
620000
625000
630000
635000
640000
645000
650000
655000
660000
665000
670000
675000
680000
685000
690000
695000
700000
705000
710000
715000
720000
725000
730

In [5]:
data['cvd_25_death_urban_est'].sum()

133824.5343292148

In [6]:
data['cvd_25_death_urban_lb'].sum()

71421.09765825965

In [7]:
data['cvd_25_death_urban_ub'].sum()

201496.5094117354

In [8]:
data['cvd_25_death_rural_est'].sum()

284825.64609822724

In [9]:
data['cvd_25_death_rural_lb'].sum()

152228.70535227712

In [10]:
data['cvd_25_death_rural_ub'].sum()

429267.0549533321

In [11]:
data.to_csv('ozone-cvd-mortality-1996(v0).csv', index=False)